# Spotify Scraping
Il seguente notebook illustra il processo di raccolta dei dati per il progetto. 
Partendo dalle tracce salvate dal mio account di Spotify, ho cercato di accumulare più artisti significativi possibili nei seguenti modi:
1. Ricorsivamente, richiedendo all'API artisti simili ad un artista.
2. Per ogni traccia pubblicata da un certo artista, venivano salvati tutti gli artisti che hanno collaborato con lui.

Per artisti significativi si indendono gli artisti con almeno 200 follower e appartenti ad almeno un genere italiano.

La raccolta dei dati è avvenuta in maniera frammentata e non in pochi blocco. Questo, a discapito dell'efficenza, ha permesso la collezione delle informazioni possibile anche senza dover eseguire tutto il codice da zero.

## Accesso a Spotify tramite API

In [1]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
import json
import time

with open('cred.json') as cred_file:
    cred = json.load(cred_file)[1]

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=cred["client_id"],
                                               client_secret=cred["client_secret"],
                                               redirect_uri=cred["redirect_uri"],
                                               scope=cred["scope"],
                                               username=cred["username"]))

## Salvataggio di tutti gli artisti de "La mia Libreria"

In [ ]:
artisti = []
limit = 50
after = 0
results = sp.current_user_saved_tracks(limit=limit, offset=after)

while True:
    results = sp.current_user_saved_tracks(limit=limit, offset=after)
    print("Limite: ", limit, ", Offset: ", after, ", Numero di risultati: ", len(results['items']))

    for item in results['items']:
        track = item["track"]
        if track["type"] == "track":
            for artist in track["artists"]:
                if not list(filter(lambda x: x["uri"] == artist["uri"], artisti)):
                    artisti.append({"uri":  artist["uri"], "name": artist["name"]})
    
    after += limit
    if len(results['items']) < limit:   
        break

Dopo aver accumulato tutti gli artisti della libreria dell'account Spotify, è stato fatto un primo filtraggio, eliminando tutti gli artisti non appartenenti ad almeno un genere italiano.

In [3]:
def divide_chunks(l, n):
    for i in range(0, len(l), n): 
        yield l[i:i + n]


def artist_filter(genre, artists):
    results = []
    artists_id = [artist["uri"] for artist in artists]


    for artists_id_chunk in divide_chunks(artists_id, 50):
        artists_more_info = sp.artists(artists_id_chunk)

        for artist in artists_more_info["artists"]:
            for music_genre in artist["genres"]:
                if genre in music_genre:
                    result = {}
                    result["uri"], result["name"], result["followers"] = artist["uri"], artist["name"], artist["followers"]["total"]
                    results.append(result)
                    break

    return results


# Filtro per artisti italiani
artisti = artist_filter("italian", artisti)

# Salvataggio dei progressi fatti
with open('artisti_liberia.json', 'w') as outfile:
    json.dump(artisti, outfile, indent=4)

In [4]:
# Checkpoint 1
with open('artisti_liberia.json') as json_file:
    artisti = json.load(json_file)

## Identificazione di artisti simili tra di loro
Tramite la funzione `sp.artist_related_artists(artista_id)`, Spotify ritorna una serie di artisti simili a quello passato per parametro. Utilizzando questa funzione ricorsivamente, si accumulano ulteriori artisti italiani.

In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

def get_artists_from_artists(artist_uri, missing_steps):
    if missing_steps > 0:
        try:
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            artisti = artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])

        partial_result = flatten([get_artists_from_artists(artista["uri"], missing_steps - 1) for artista in artisti])
    
        return artisti + partial_result
    else:
        try:
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])
        except Exception as e:
            print(e)
            time.sleep(5)
            return artist_filter("italian", sp.artist_related_artists(artist_uri)["artists"])


recursive_artist = get_artists_from_artists(artisti[0]["uri"], 2)

# Eliminazione degli artisti che risulano duplicati nella lista
recursive_artist_no_dupl = []
for x in recursive_artist:
    if x not in recursive_artist_no_dupl:
        recursive_artist_no_dupl.append(x)

with open('artisti_ricorsivi.json', 'w') as json_file:
    json.dump(recursive_artist_no_dupl, json_file, indent=4)
    
len(recursive_artist_no_dupl)

In [12]:
# Checkpoint 2
with open('artisti_ricorsivi.json', 'r') as json_file:
    recursive_artists = json.load(json_file)

## Collezione di artisti basata sulle collaborazioni

In [ ]:
artists_full = {}

for counter, artist in enumerate(recursive_artists):
    results = sp.artist_albums(artist["uri"], album_type='album')
    albums = results['items']
    
    while results['next']:
        results = sp.next(results)
        albums.extend(results['items'])

    for album in albums:
        album_tracks = sp.album_tracks(album["id"], limit=50)
        for track in album_tracks["items"]:
            for track_artist in track["artists"]:
                artists_full[track_artist["name"]] = {
                    "id": track_artist["id"]
                }
    print(f"{counter}/{len(recursive_artists)} artisti caricati")
    time.sleep(3)

print(artists_full)

In [26]:
len(artists_full)
with open('artisti_full.json', 'w') as json_file:
    json.dump(artists_full, json_file, indent=4)

Teniamo solo gli artisti con almeno 200 follower e filtriamo solo per artisti italiani

## Filtraggio dei risultati
A questo punto gli artisti con meno di 200 followers e che non appartengono ad almeno un genere italiani vengono scartati.

In [46]:
artisti_filtrati = {}

for artist_name, artist_id in artists_full.items():
    artist_data = sp.artist(f"spotify:artist:{artist_id['id']}")
    
    for music_genre in artist_data["genres"]:
        if "italian" in music_genre:
            if artist_data["followers"]["total"] > 200:
                artisti_filtrati[artist_name] = artist_id
                artisti_filtrati[artist_name]["followers"] = artist_data["followers"]["total"]
                break

In [5]:
with open('artisti_filtrati.json', 'r') as json_file:
    artisti_filtrati = json.load(json_file)

## Scraper album/tracks
A questo punto, per ogni artista, vengono salvati tutti le canzoni e gli album pubblicati

In [ ]:
artists_albums_tracks = {}

for i, artist_info in enumerate(artisti_filtrati.items()):
    artist, info = artist_info
    print(f"Artista {i+1} su {len(artisti_filtrati)}")
    albums = sp.artist_albums(f"spotify:artist:{info['id']}")

    albums_to_add = {}
    for album in albums["items"]:
        albums_to_add[album["id"]] = {
            "name": album["name"], 
            "release_date": album["release_date"], 
            "total_tracks": album["total_tracks"],
            "type": album["type"]
        }

        tracks_to_add = {}
        for track in sp.album_tracks(album["id"], limit=50)["items"]:
            tracks_to_add[track["id"]] = {
                "name": track["name"], 
                "duration_ms": track["duration_ms"], 
                "explicit": track["explicit"],
                "type": track["type"]
            }
        
        albums_to_add[album["id"]]["tracks"] = tracks_to_add
        artists_albums_tracks[info["id"]] = {
            "name": artist,
            "followers": info["followers"],
            "albums": albums_to_add,
        }
    
    time.sleep(0.5)


In [13]:
with open('artists_albums_tracks.json', 'w') as json_file:
    json.dump(artists_albums_tracks, json_file, indent=4)

In [2]:
with open('artists_albums_tracks.json', 'r') as json_file:
    artists_albums_tracks = json.load(json_file)

## Scraper canzoni
Tramite A-Z Lyrics si scaricano i testi di ogni canzone collezionata precendetemente.

In [10]:
from azlyrics.azlyrics import lyrics
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import copy

def get_clean_lyrics(text):
    text_tokens = word_tokenize(text)
    tokens_without_sw = [word.lower() for word in text_tokens if not word in stopwords.words("italian")]
    clean_tokens = [word for word in tokens_without_sw if re.findall(r"[^-!$%^&*()_+|~=`{}\[\]:\";'<>?,.\/]", word)]
    return " ".join(sorted(set(clean_tokens), key=clean_tokens.index))


artists_albums_tracks_lyrics = copy.deepcopy(artists_albums_tracks)

for artist, artist_info in artists_albums_tracks.items():
    for album, album_info in artist_info["albums"].items():
        for track, track_info in album_info["tracks"].items():
            if "Live" not in track_info["name"]:
                track_name = track_info["name"].split("feat.")[0].lower().replace("(", "").replace(")", "").replace(" ", "").replace("-","")
                track_name = "".join(re.findall(r"[a-z0-9]+", track_name))
                print(track_name)
                lyr = lyrics(artist_info["name"], track_name)
                if isinstance(lyr, dict):
                    print(lyr)
                    continue

                artists_albums_tracks_lyrics[artist]["albums"][album]["tracks"][track]["lyrics"] = get_clean_lyrics(lyr[0])
                time.sleep(5)
            

antipatico
charlesmansonbuonnatale2
90min
donmedelln
mictaser
intro
{'Error': 'Unable to find intro by salmo'}
morteindirettayazeeremix
{'Error': 'Unable to find morteindirettayazeeremix by salmo'}
yokoonodjoverremix
{'Error': 'Unable to find yokoonodjoverremix by salmo'}
laprimavoltaunotherremix
borderline
hellcome
morteindiretta
lacanzonenostraconblancosalmo
{'Error': 'Unable to find lacanzonenostraconblancosalmo by salmo'}
90min
perdonami
estatedimmerda
donmedelln
giuda
iosonoqui
1984
mussoleeni
inri
alfaalfa
alfaalfa
frombktorm
intro
tajerino
introducingnoyznarcos
guardamiadessoprodsine
{'Error': 'Unable to find guardamiadessoprodsine by noyznarcos'}
wildboyspt2
{'Error': 'Unable to find wildboyspt2 by noyznarcos'}
sciacalli
sinnmemoro
{'Error': 'Unable to find sinnmemoro by noyznarcos'}
dopegames
{'Error': 'Unable to find dopegames by noyznarcos'}
lobo
trainingday
gameover
wildboys
{'Error': 'Unable to find wildboys by noyznarcos'}
dragyoutohell
{'Error': 'Unable to find dragyouto

KeyboardInterrupt: 

In [11]:
with open('artists_albums_tracks_lyrics.json', 'w') as json_file:
    json.dump(artists_albums_tracks_lyrics, json_file, indent=4)

In [ ]:
with open('artists_albums_tracks_lyrics.json', 'r') as json_file:
    artists_albums_tracks_lyrics = json.load(json_file)

In [ ]:
artists_albums_tracks_lyrics